In [3]:
import json
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import os
import pickle
from tqdm import tqdm
import lightning as L
import torch.nn as nn
from lightning.pytorch.callbacks import ModelCheckpoint
import sys
import random

sys.path.append("/home/maxihuber/eeg-foundation/")

## Task List

Clinical:

- /itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/age.json

  'train', 'test'
  {'input': [], 'output': float, 'start': int, 'length': int} -> Regression
  => 0:-1 always
  
- /itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/cli_depression.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int} -> Classification
  => 0:-1 always
  out_dim = 2
  outputs = {'No Depression', 'Depression'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/cli_parkinsons.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int} -> Classification
  => 0:-1 always
  out_dim = 2
  outputs = {'No PD', 'PD'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/cli_schizophrenia.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int} -> Classification
  => 0:-1 always
  out_dim = 2
  outputs = {'No Schizophrenia', 'Schizophrenia'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/sex.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int} -> Classification
  => 0:-1 always
  out_dim = 2
  outputs = {'F', 'M'}

Motor imagery: /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/mi_all.json

- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/eye_open_closed.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'eye open', 'eye closed'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/eye_vh.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'vertical', 'horizontal'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/flexion_extension_imaginary.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'hand movement imagined elbow flexion', 'hand movement imagined elbow extension'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/flexion_extension_real.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'hand movement elbow extension', 'hand movement elbow flexion'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/grasp_real.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'movement palmar grasp', 'movement lateral grasp'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/lr_imaginary.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'left hand imagined movement', 'right hand imagined movement'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/lr_real.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'right hand movement', 'left hand movement'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/mi_task_body_parts.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 4
  outputs = {'rest', 'right hand movement', 'foot movement', 'left hand movement'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/pronation_supination_real.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'movement supination', 'movement pronation'}

- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/pronation_supination_imaginary.json

  'train', 'test'
  {'input': [], 'output': str, 'start': int, 'length': int, 'length_seconds': float} -> Classification
  out_dim = 2
  outputs = {'imagined supination', 'imagined pronation'}
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/fingers_imaginary.json (empty)

Error-Related Potential:
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/erp_all.json

  'train', 'test'
  {'input': [], 'label': str, 'length_seconds': float, 'start': int, 'end': int} -> Classification
  out_dim = 5
  
- /itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/errp_all.json

  'train', 'test'
  {'input': [], 'label': str, 'length_seconds': float, 'start': int, 'end': int} -> Classification

EyeNet:
- /itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Amp_*.json

  _train, _val, _test files
  {'input': [], 'output': {'amp': float, 'ang': float}, start: int, 'length': int} -> Regression (1d, amplitude)

- /itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Ang_*.json

  _train, _val, _test files
  {'input': [], 'output': {'amp': float, 'ang': float}, start: int, 'length': int} -> Regression (1d, angle)
  
- /itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_*.json

  _train, _val, _test files
  {'input': [], 'output': {'label_LR': str}, start: int, 'length': int} -> Classification
  
- /itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Position_*.json

  _train, _val, _test files
  {'input': [], 'output': {'x-pos': float, 'y-pos': float}, start: int, 'length': int} -> Regression
  use EuclideanDistance

## Channel information for each task

In [12]:
def load_index(data_index_path):
    with open(data_index_path, 'r') as f:
        train_test_dict = json.load(f)
    train_samples = train_test_dict['train']
    test_samples = train_test_dict['test']
    return train_samples, test_samples

class_name = "Clinical"
task_name = "Age"
task_type = "Regression"
time_col = "Time in Seconds"
json_path = "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/age.json"
task_channels = set(['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5',
       'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6',
       'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7',
       'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3',
       'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CP4', 'TP8',
       'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8', 'VEOG', 'X', 'Y',
       'Z'])

class_name = "Clinical"
task_name = "Depression"
task_type = "Classification"
time_col = "Time in Seconds"
json_path = "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/cli_depression.json"
#task_channels = set()

prefix_filepath = "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_prepared/"

train_index, test_index = load_index(json_path)
train_index = [train_index[0]]

def load_file_data(data_index, task_channels):
    num_samples = 0
    data = {}
    outputs = {}
    srs = {}
    durs = {}
    channels = {}
    for sample in tqdm(data_index, desc="Loading data"):
        # Load and concatenate dataframe
        input_files = sample["input"]
        df = pd.DataFrame()
        for file in input_files:
            file = prefix_filepath + file
            with open(file, 'rb') as f:
                df_new = pickle.load(f)
                df = pd.concat([df, df_new], axis=0)
        start, length = int(sample["start"]), int(sample["length"])
        df = df.iloc[start:length, :]
        # Add metadata
        sr = int(1 / float(float(df[time_col].iloc[1]) - float(df[time_col].iloc[0])))
        outputs[num_samples] = sample["output"]
        srs[num_samples] = sr
        durs[num_samples] = len(df) / sr
        channels[num_samples] = set(df.columns) & task_channels
        data[num_samples] = df
        num_samples += 1
    return data, outputs, srs, durs, channels

train_data, train_outputs, train_sr, train_dur, train_channels = load_file_data(train_index, task_channels)

206 33


Loading data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:11<00:00, 71.86s/it]


In [119]:
print(train_index.keys())
for k, v in train_index['EEGEyeNet_Position_train'][0].items():
    print(f"{k}: {type(v)} = {v[0] if type(v) == list else v}")

dict_keys(['EEGEyeNet_Position_train', 'task_type'])
input: <class 'list'> = /itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-EP10-EP10_DOTS1_EEG-segment-000000002-fixation.pickle
output: <class 'dict'> = {'label_x_position': 408.1, 'label_y_position': 315.1}
start: <class 'int'> = 137
length: <class 'int'> = 500


In [12]:
sample = train_index['train'][0]
df = pd.DataFrame()
for file in sample['input']:
    file = '/itet-stor/maxihuber/deepeye_storage/foundation_clinical_prepared/' + file
    with open(file, 'rb') as f:
        df_new = pd.read_pickle(f)
        df = pd.concat([df, df_new], axis=0)
print(df.shape)

(90932, 90)


In [18]:
cli_channel_names_path = '/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/dataset_channel_name_mapping.json'
with open(cli_channel_names_path, 'r') as f:
    cli_channel_names = json.load(f)["Channels"][0]

all_channels = []
for k, v in cli_channel_names.items():
    all_channels += v
all_channels = list(set(all_channels))
print(sorted(all_channels))

['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPz', 'Cz', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCz', 'FT7', 'FT8', 'Fp1', 'Fp2', 'Fz', 'Mastoids', 'O1', 'O2', 'Oz', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO3', 'PO4', 'PO7', 'PO8', 'POz', 'Pz', 'T7', 'T8', 'TP7', 'TP8', 'Veog', 'X', 'Y', 'Z']


## Split Eye Direction Task

In [25]:
stage = 'test'

with open(f'/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_{stage}.json', 'r') as f:
    eye_full = json.load(f)

task_name = list(eye_full.keys())[0]
task_type = list(eye_full.keys())[1]
samples = eye_full[task_name]

samples_amplitude = [{'input': sample['input'],
                      'output': {'label_amplitude': sample['output']['label_amplitude']},
                      'start': sample['start'],
                      'length': sample['length'],
                     } for sample in samples]
samples_angle = [{'input': sample['input'],
                  'output': {'label_angle': sample['output']['label_angle']},
                  'start': sample['start'],
                  'length': sample['length'],
                 } for sample in samples]

print(samples[0])
print(samples_amplitude[0])
print(samples_angle[0])

amp_full = {task_name: samples_amplitude, "task_type": eye_full[task_type]}
ang_full = {task_name: samples_angle, "task_type": eye_full[task_type]}

with open(f'/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Amp_{stage}.json', 'w') as f:
    json.dump(amp_full, f)

with open(f'/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Ang_{stage}.json', 'w') as f:
    json.dump(ang_full, f)

{'input': ['/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-ep57-ep57_DOTS1_EEG-segment-000000003-fixation.pickle', '/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-ep57-ep57_DOTS1_EEG-segment-000000004-saccade.pickle', '/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-ep57-ep57_DOTS1_EEG-segment-000000005-fixation.pickle', '/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-ep57-ep57_DOTS1_EEG-segment-000000006-saccade.pickle', '/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-ep57-ep57_DOTS1_EEG-segment-000000007-fixation.pickle'], 'output': {'label_amplitude': 306.0, 'label_angle': 0.6435011087932844}, 'start': 840, 'length': 500}
{'input': ['/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synchronised_min-ep57-ep57_DOTS1_EEG-segment-000000003-fixation.pickle', '/itet-stor/kard/deepeye_storage/eegeyenet_prepared/dots_data-synch